# Getting Started with Julia

These instructions are mostly just a summary of the information included on

https://lectures.quantecon.org/jl/getting_started.html

Please see that page for more details

## Installation

Go to https://julialang.org/downloads/ and download the Julia 0.6 which corresponds to your operating system

Once you have downloaded this file, follow the operating system specific instructions at https://julialang.org/downloads/platform.html

**If you run into any trouble, please raise your hand and ask for help!**

## Packages

Julia comes with an automated package management system.

### How does it work?

Packages register themselves with the [higher power](https://github.com/tkelman).

In order to become a registered package, some basic requirements such as tests, description of dependencies, and a descriptive `README` file.

### Install and Uninstall (Registered) Packages

Installation via `Pkg.add("PackageName")`

Uninstall via `Pkg.rm("PackageName")`

### Useful Packages

A list of all registered packages (ordered by how many github star they have) can be found at: https://juliaobserver.com/packages

Packages that we find useful

* [BasisMatrices.jl](https://github.com/QuantEcon/BasisMatrices.jl): Interpolation based on CompEcon library by Miranda and Fackler
* [Distributions.jl](https://github.com/JuliaStats/Distributions.jl): Package for interfacing with various probability distributions
* [Interpolations.jl](https://github.com/JuliaMath/Interpolations.jl): Interpolation library written by programmers in image processing field
* [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl): A pure Julia version of many optimization routines
* [Plots.jl](https://github.com/JuliaPlots/Plots.jl): An abstract plotting package which interfaces with various plotting backends
* [QuantEcon](https://github.com/QuantEcon/QuantEcon.jl): Contains many tools relevant to economists

### Exercise

Install the following packages:

* `Distributions`
* `IJulia`
* `Interpolations`
* `KernelDensity`
* `PlotlyJS`
* `PyPlot`
* `Plots`
* `QuantEcon`

We will use these packages for both Victoria and my presentations.

## Julia Types

You should frequently define your own types -- Julia's JIT compilation helps ensure that user defined types will have same performance as the base types

Key feature of Julia is multiple dispatch -- Often allows writing code in a generic way.

Example using stochastic processes below

In [ ]:
abstract type Exog end

"""
This type holds parameters for an AR(1) of the form

y_{t+1} = rho * y_{t} + sigma * eps

where eps ~ N(0, 1)
"""
struct AR1 <: Exog
    rho::Float64  # you should put types on the fields of your types
    sigma::Float64
end

"""
This type holds a Markov chain which consists of

* stochastic matrix (Π)
* state values (vals)
* intial distribution (x0)
"""
struct MarkovChain{T} <: Exog
    Π::Matrix{Float64}
    vals::Vector{T}
    x0::Vector{Float64}
end

# functions to give `vals` and `x0` default arguments
MarkovChain(Π, v) = MarkovChain(Π, v, fill(1/length(v), length(v)))
MarkovChain(Π) = MarkovChain(Π, 1:size(Π, 1))

In [ ]:
iter(ar1::AR1, x) = ar1.rho*x + ar1.sigma*randn()

function iter(mc::MarkovChain, s::Int)
    ind = searchsortedfirst(cumsum(vec(mc.Π[s, :])), rand())
    return mc.vals[ind]
end

iter{T}(mc::MarkovChain{T}, v::T) = iter(mc, findfirst(mc.vals, v))
iter(::Exog, x) = error("iter should be implemented by each Exog subtype")

state_type(ar1::AR1) = Float64
state_type(mc::MarkovChain) = eltype(mc.vals)

In [ ]:
function simulate{T<:Exog}(s::T, x0; capT=1000)
    # Allocate memory
    out = Array{state_type(s)}(capT)

    # Simulate
    out[1] = x0
    for t=2:capT
        out[t] = iter(s, out[t-1])
    end

    return out
end

In [ ]:
ar1 = AR1(0.9, 0.1)

@time simulate(ar1, 0.0; capT=10_000_000);

In [ ]:
mc = MarkovChain([0.75 0.25; 0.25 0.75], [1.0, 2.0])

@time simulate(mc, mc.vals[1]; capT=10_000_000);

## Misc Interesting Links

* Tom Sargent [JuliaCon keynote](https://www.youtube.com/watch?v=KkKBwJkYgVk&t) on why economists should be interested in Julia
* NYFed [blogpost](http://libertystreeteconomics.newyorkfed.org/2017/05/forecasting-with-julia.html) on moving their [forecasting models](https://github.com/FRBNY-DSGE/DSGE-2014-Sep) from Matlab to Julia
* Brad Setzler's [tutorials](https://juliaeconomics.com/) on Julia for economists
